In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT

import itertools

def genState(vars,s,i,n):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),BVType(n+1))
    
    return state


def init(state,n):
    bvmax = BV(2**n,n+1)
    return And(BVUGT(state['x'],BVZero(n+1)),
               BVULT(state['x'],bvmax),
               BVUGT(state['y'],BVZero(n+1)),
               BVULT(state['y'],bvmax),
               Equals(state['z'],BVZero(n+1)),
               Equals(state['pc'],BVZero(n+1)))


def trans(curr,prox,n):
    
    bv0 = BVZero(n+1)
    bv1 = BVOne(n+1)
    bv2 = BV(2,n+1)
    bvmax = BV(2**n,n+1)
    
    t0 = And(Equals(curr['pc'],bv0),
             Equals(prox['pc'],bv1),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z'])) 
    
    t1 = And(Equals(curr['pc'],bv1),
             Equals(BVAdd(curr['y'],bv1),BVXor(curr['y'],bv1)),
             Not(Equals(curr['y'],bv0)),
             Equals(prox['pc'],BV(2,n+1)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    t2 = And(Equals(curr['pc'],BV(2,n+1)),
             Equals(prox['pc'],BV(1,n+1)),
             BVUGT(bvmax,prox['x']),
             Equals(BVMul(curr['x'],bv2),prox['x']),
             Equals(BVUDiv(curr['y'],bv2),prox['y']),
             Equals(curr['z'],prox['z']))
    
    te = And(Equals(curr['pc'],BV(2,n+1)),
             Equals(prox['pc'],BV(5,n+1)),
             BVUGT(BVMul(curr['x'],bv2),bvmax),
             Equals(BVMul(curr['x'],bv2),prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    te2 = And(Equals(curr['pc'],BV(5,n+1)),
             Equals(prox['pc'],BV(5,n+1)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    t3 = And(Equals(curr['pc'],BV(1,n+1)),
             Equals(BVSub(curr['y'],bv1),BVXor(curr['y'],bv1)),
             Not(Equals(prox['y'],bv0)),
             Equals(prox['pc'],BV(3,n+1)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    t4 = And(Equals(curr['pc'],BV(3,n+1)),
             Equals(prox['pc'],BV(1,n+1)),
             Equals(curr['x'],prox['x']),
             Equals(BVSub(curr['y'],BVOne(n+1)),prox['y']),
             Equals(curr['z']+curr['x'],prox['z']),
             BVUGT(bvmax,prox['z']))
    
    te3 = And(Equals(curr['pc'],BV(3,n+1)),
             Equals(prox['pc'],BV(5,n+1)),
             Equals(curr['x'],prox['x']),
             Equals(BVSub(curr['y'],BVOne(n+1)),prox['y']),
             Equals(curr['z']+curr['x'],prox['z']),
             BVUGT(curr['x']+curr['z'],bvmax))
    
    t5 = And(Equals(curr['pc'],BV(1,n+1)),
             Equals(prox['pc'],BV(4,n+1)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']),
             Equals(curr['y'],bv0))
    
    t6 = And(Equals(curr['pc'],BV(4,n+1)),
             Equals(prox['pc'],BV(4,n+1)),
             Equals(curr['x'],prox['x']),
             Equals(curr['y'],prox['y']),
             Equals(curr['z'],prox['z']))
    
    return Or(t1,t2,t3,t0,t4,t5,t6,te,te2,te3)



def error(state,n):
    m = n+1
    bvmax = BV(2**n,n+1)
    bvmaxx = BV(2**n+1,n+1)
    
    t1 = And(Equals(state['pc'],BV(5,n+1)),BVUGE(state['x'],bvmax),BVUGE(state['y'],BVZero(n+1)),
             BVUGE(state['z'],BVZero(n+1)),BVULE(state['x'],bvmaxx),BVULE(state['y'],bvmax),BVULE(state['z'],bvmax))
    t2 = And(Equals(state['pc'],BV(5,n+1)),BVUGE(state['x'],BVZero(n+1)),BVUGE(state['y'],BVZero(n+1)),
             BVUGE(state['z'],bvmax),BVULE(state['z'],bvmaxx),BVULE(state['x'],bvmax),BVULE(state['y'],bvmax))
    
    return Or(t1,t2)






def genTrace(vars,init,trans,error,k,n):
    with Solver(name="z3") as s:
        
        
        X = [genState(vars,'X',i,k) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0],k)
        Tks = [ trans(X[i],X[i+1],k) for i in range(n) ]
        
        if s.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))



In [2]:
genTrace(['pc','x','y','z'],init,trans,error,10,40)

Estado: 0
           pc = 0_11
           x = 33_11
           y = 156_11
           z = 0_11
Estado: 1
           pc = 1_11
           x = 33_11
           y = 156_11
           z = 0_11
Estado: 2
           pc = 2_11
           x = 33_11
           y = 156_11
           z = 0_11
Estado: 3
           pc = 1_11
           x = 66_11
           y = 78_11
           z = 0_11
Estado: 4
           pc = 2_11
           x = 66_11
           y = 78_11
           z = 0_11
Estado: 5
           pc = 1_11
           x = 132_11
           y = 39_11
           z = 0_11
Estado: 6
           pc = 3_11
           x = 132_11
           y = 39_11
           z = 0_11
Estado: 7
           pc = 1_11
           x = 132_11
           y = 38_11
           z = 132_11
Estado: 8
           pc = 2_11
           x = 132_11
           y = 38_11
           z = 132_11
Estado: 9
           pc = 1_11
           x = 264_11
           y = 19_11
           z = 132_11
Estado: 10
           pc = 3_11
           x = 264_11
  

In [3]:
def invert(trans):
    return (lambda c,p,t: trans(p,c,t))

def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

In [5]:
def model_checking(vars,init,trans,error,N,M,k):
    with Solver(name="z3") as s:
        
        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i,k) for i in range(N+1)]
        Y = [genState(vars,'Y',i,k) for i in range(M+1)]
        
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        #order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1]) 
        n = 1
        m = 1
        while True:
            
            Tn = And([trans(X[i],X[i+1],k) for i in range(n)])
            I = init(X[0],k)
            Rn = And(I,Tn)
            
            Bm = And([invert(trans)(Y[i],Y[i+1],k) for i in range(m)])
            E = error(Y[0],k)
            Um = And(E,Bm)
            
            Vnm = And(Rn,same(X[n],Y[m]),Um)
            
            
            
            if s.solve([Vnm]):
                print("unsafe")
                return
            else:
                C = binary_interpolant(And(Rn,same(X[n],Y[m])),Um)
                if C is None:
                    print("interpolante none")
                    break
                C0 = rename(C,X[0])
                C1 = rename(C,X[1])
                T = trans(X[0],X[1],k)
                
                if not s.solve([C0,T,Not(C1)]):
                    print("safe")
                    return
                else:
                    S = rename(C,X[n])
                    while True:
                        A = And(S,trans(X[n],Y[m],k))
                        if s.solve([A,Um]):
                            print("nao é possivel majorar")
                            break
                        else:
                            Cnew = binary_interpolant(A,Um)
                            Cn = rename(Cnew,X[n])
                            if s.solve([Cn,Not(S)]):
                                S = Or(S,Cn)
                            else:
                                print("safe")
                                return

#####

model_checking(['pc','x','y','z'], init, trans, error, 50, 50,10)  

nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
nao é possivel majorar
unsafe
